# Python列表进阶与异常处理 - Lec08_B.pdf（信息导论课）
本笔记基于Lec08_B.pdf课件，梳理**列表迭代修改问题、深浅拷贝、列表推导式、异常处理与断言**核心知识点，针对课件中的难点（如迭代器机制、深浅拷贝底层差异、异常与断言的适用场景）补充额外解释.


## 1. 课程通知：Homework 2提交信息（对应课件🔶2-4至）
课件开篇明确作业核心要求，需重点关注提交细节（逾期可能导致成绩无效）：
- **题目获取**：通过Blackboard（BB）平台，进入“作业”板块，找到“Programming Assignment 2”查看具体指令与问题。
- **提交渠道**：仅支持OJ（Online Judge）系统提交，需确保代码格式符合OJ要求（如无多余打印、输入输出匹配）。
- **截止时间**：10月23日晚9点（21:00），建议提前1-2小时提交，避免网络延迟或格式问题导致逾期。

【额外拓展】：OJ系统通常会实时反馈代码通过率（如“编译错误”“答案错误”“运行超时”），提交前可在本地用课件中的测试案例（如`remove_dups`的输入输出）验证，减少反复提交次数。

In [ ]:
# 作业提交提醒工具（含本地测试建议）
def homework_2_guide():
    print("="*50)
    print("Homework 2 提交指南")
    print("="*50)
    print("1. 题目查看：BB → 作业 → Programming Assignment 2")
    print("2. 提交平台：OJ系统（需登录校园账号）")
    print("3. 截止时间：10月23日 21:00（建议提前提交）")
    print("4. 本地测试建议：")
    print("   - 用课件示例数据验证（如remove_dups的[10,20,30,40]案例）")
    print("   - 检查输入输出格式（如是否需要打印额外提示语）")
    print("   - 测试边界情况（如空列表、单元素列表）")

homework_2_guide()

## 2. 列表删除操作回顾（对应课件🔶2-12至）
课件延续Lec07内容，重申列表三种核心删除方法的区别——均**直接修改原列表**（Mutable特性），需注意返回值与错误场景：
- **`del L[index]`**：按索引删除，无返回值；若索引超出列表长度（如`L=[1,2], del L[3]`），抛出`IndexError`。
- **`L.pop(index)`**：按索引删除（默认删除最后一个元素），**返回被删除元素**；索引越界同样抛出`IndexError`，适合需获取删除元素的场景（如栈的“弹出”操作）。
- **`L.remove(element)`**：按元素值删除，仅删除**第一个匹配元素**，无返回值；若元素不在列表中（如`L=[1,2], L.remove(3)`），抛出`ValueError`。

课件经典示例：对初始列表`L = [2,1,3,6,3,7,0]`依次执行`remove(2)`、`remove(3)`、`del L[1]`、`L.pop()`，最终结果为`[1,3,7]`（需严格按顺序执行，因每次操作都会改变列表结构）。

【额外拓展】：`L.remove(element)`的时间复杂度为O(n)（需遍历列表查找第一个匹配元素），若需删除列表中所有某元素，频繁调用`remove`会导致O(n²)的低效（如`remove_all`的错误for循环写法），推荐用列表推导式或while循环优化（课件后续`remove_all`的正确写法）。

In [ ]:
# 列表删除操作对比与效率说明（对应课件🔶2-12至）
# 初始列表（课件示例）
L = [2, 1, 3, 6, 3, 7, 0]
print("初始列表:", L)  # 输出：[2, 1, 3, 6, 3, 7, 0]

# 1. L.remove(2)：按值删除第一个匹配元素（O(n)时间）
L.remove(2)
print("1. remove(2)后:", L)  # 输出：[1, 3, 6, 3, 7, 0]

# 2. L.remove(3)：再次按值删除（需重新遍历列表）
L.remove(3)
print("2. remove(3)后:", L)  # 输出：[1, 6, 3, 7, 0]

# 3. del L[1]：按索引删除（O(1)定位，后续元素前移O(n)）
del L[1]
print("3. del L[1]后:", L)  # 输出：[1, 3, 7, 0]

# 4. L.pop()：删除最后一个元素（O(1)时间，无需遍历）
deleted_elem = L.pop()
print("4. pop()删除的元素:", deleted_elem)  # 输出：0
print("4. pop()后最终列表:", L)  # 输出：[1, 3, 7]

# 效率对比：删除最后一个元素用pop()更高效
import time
big_list = list(range(100000))  # 10万元素列表

# 测试pop()删除最后一个元素（O(1)）
start = time.time()
big_list.pop()
end = time.time()
print(f"\npop()删除最后一个元素耗时: {end - start:.6f}秒")

# 测试remove(99999)删除最后一个元素（O(n)）
big_list = list(range(100000))
start = time.time()
big_list.remove(99999)
end = time.time()
print(f"remove()删除最后一个元素耗时: {end - start:.6f}秒")  # 耗时显著更长

## 3. 迭代中修改列表的Tricky案例（对应课件🔶2-112至）
课件重点分析“迭代时修改列表导致的逻辑漏洞”，核心案例为`remove_dups`函数——需求是“修改`L1`，删除所有同时存在于`L2`的元素”（如`L1=[10,20,30,40]`、`L2=[10,20,50,60]`，期望结果`[30,40]`）。这是课件的核心难点，需理解Python迭代器的底层机制。

### 3.1 错误实现：直接迭代原列表（导致元素跳过）
错误写法的问题根源：**Python的for循环本质是通过“迭代器（iterator）”实现的，迭代器会用内部计数器跟踪当前索引，修改列表长度后计数器不更新，导致跳过后续元素**。

In [ ]:
# 错误实现：迭代原列表导致元素跳过（对应课件🔶2-112至）
def remove_dups_wrong(L1, L2):
    """错误写法：迭代L1时删除元素，内部计数器不更新"""
    print("迭代过程跟踪：")
    for idx, e in enumerate(L1):  # 用enumerate显示迭代索引（课件未显式提及，拓展用）
        print(f"  计数器={idx}, 当前元素={e}, 此时L1={L1}")
        if e in L2:
            L1.remove(e)
            print(f"  → 删除{e}，L1变为={L1}")

# 测试（课件示例数据）
L1 = [10, 20, 30, 40]
L2 = [10, 20, 50, 60]
print("初始L1:", L1)
remove_dups_wrong(L1, L2)
print("错误实现最终结果（期望[30,40]）:", L1)  # 输出：[20, 30, 40]

【额外拓展】：迭代器工作原理拆解：
1. for循环开始时，调用`iter(L1)`生成迭代器，迭代器记录列表的初始状态（长度、元素地址）。
2. 每次循环调用`next(迭代器)`，返回当前索引的元素，然后计数器+1（固定递增，不随列表修改调整）。
3. 当删除元素`10`后，L1变为`[20,30,40]`，但计数器仍按原计划从0→1，此时`next(迭代器)`返回L1[1]=30，跳过了20。


```python
# 正确实现：迭代克隆列表，修改原列表（对应课件🔶2-151至）
import copy

def remove_dups_correct(L1, L2, clone_method="slice"):
    """
    正确写法：克隆L1后迭代，修改原列表
    clone_method: "slice"（L1[:]）、"list"（list(L1)）、"copy"（copy.copy(L1)）
    """
    # 三种克隆方式（等价，课件用L1[:]）
    if clone_method == "slice":
        L1_copy = L1[:]
    elif clone_method == "list":
        L1_copy = list(L1)
    elif clone_method == "copy":
        L1_copy = copy.copy(L1)
    else:
        raise ValueError("clone_method仅支持slice/list/copy")
    
    print("\n正确实现迭代过程（迭代克隆列表）：")
    for e in L1_copy:
        print(f"  检查克隆列表元素={e}，原列表当前={L1}")
        if e in L2:
            L1.remove(e)
            print(f"  → 删除{e}，原列表变为={L1}")

# 测试（三种克隆方式均有效）
L1 = [10, 20, 30, 40]
L2 = [10, 20, 50, 60]
remove_dups_correct(L1, L2, "slice")
print("正确实现最终结果:", L1)  # 输出：[30, 40]（符合期望）

### 3.3 常见误区：`L1_copy = L1`是别名，非克隆（对应课件🔶2-168至）
课件明确指出：`=`赋值仅创建“别名（alias）”——新变量与原变量指向**同一内存对象**，而非生成新列表。迭代别名等同于迭代原列表，仍会出现“元素跳过”问题，这是初学者最易混淆的概念之一。

【额外拓展】：用`id()`函数验证“别名”与“克隆”的内存差异（`id()`返回对象的唯一内存地址，相同则为同一对象）：

In [ ]:
# 验证别名与克隆的内存差异（对应课件🔶2-168至）
L1 = [10, 20, 30, 40]

# 1. 别名：L1_copy与L1指向同一对象
L1_alias = L1
print(f"L1的内存地址: {id(L1)}")
print(f"L1_alias的内存地址: {id(L1_alias)}")
print(f"是否为同一对象（别名）: {id(L1) == id(L1_alias)}")  # 输出：True

# 2. 克隆：L1_clone与L1指向不同对象
L1_clone = L1[:]
print(f"\nL1_clone的内存地址: {id(L1_clone)}")
print(f"是否为同一对象（克隆）: {id(L1) == id(L1_clone)}")  # 输出：False

# 修改别名：影响原列表
L1_alias.append(50)
print(f"\n修改别名后L1: {L1}")  # 输出：[10,20,30,40,50]（原列表被修改）

# 修改克隆：不影响原列表
L1_clone.append(60)
print(f"修改克隆后L1: {L1}")  # 输出：[10,20,30,40,50]（原列表无变化）

## 4. 浅拷贝与深拷贝（对应课件🔶2-270至）
当列表包含**可变元素（如嵌套列表、字典）** 时，普通克隆（`L[:]`）无法复制嵌套元素的内存地址，导致“修改嵌套元素仍会影响原列表”——这是课件的另一核心难点，需通过`copy`模块区分“浅拷贝”与“深拷贝”。

### 4.1 核心概念对比（课件重点）
| 操作                | 本质                     | 适用场景                     | 嵌套元素处理               |
|---------------------|--------------------------|------------------------------|----------------------------|
| `new = old`         | 别名（共享同一对象）     | 无需复制，仅需多变量引用     | 完全共享，修改任一均影响   |
| `new = copy.copy(old)` | 浅拷贝（仅复制顶层结构） | 列表元素均为不可变类型（int、str、tuple） | 嵌套元素仍为别名           |
| `new = copy.deepcopy(old)` | 深拷贝（复制所有层级） | 列表包含可变嵌套元素（如`[[1,2], [3,4]]`） | 嵌套元素也生成新对象，完全独立 |

【额外拓展】：浅拷贝对“不可变嵌套元素”的特殊处理——若嵌套元素是不可变类型（如元组`(1,2)`），即使浅拷贝，修改嵌套元素的尝试会直接报错（因元组不可变），因此无需深拷贝。例如：
```python
old = [(1,2), (3,4)]  # 嵌套元组（不可变）
new_shallow = copy.copy(old)
# new_shallow[0][0] = 5  # 报错：TypeError: 'tuple' object does not support item assignment


```python
# 浅拷贝与深拷贝完整对比（对应课件🔶2-270至）
import copy

# 测试数据：含可变嵌套元素（嵌套列表）
old_list = [[1, 2], [3, 4], [5, "foo"]]
print("初始原列表:", old_list)

# 1. 赋值（别名）：完全共享
new_alias = old_list
new_alias[2][1] = 6  # 修改嵌套元素
print("\n1. 赋值（别名）:")
print(f"  原列表: {old_list}")  # 输出：[[1,2],[3,4],[5,6]]（被修改）
print(f"  新列表: {new_alias}")  # 输出：[[1,2],[3,4],[5,6]]

# 重置原列表
old_list = [[1, 2], [3, 4], [5, 6]]

# 2. 浅拷贝（copy.copy / 切片）：顶层复制，嵌套共享
new_shallow = copy.copy(old_list)  # 等同于 new_shallow = old_list[:]
# 修改顶层元素（原列表不受影响）
old_list.append([7, 8])
# 修改嵌套元素（新列表受影响，因嵌套列表是别名）
old_list[1][1] = 9
print("\n2. 浅拷贝:")
print(f"  原列表: {old_list}")  # 输出：[[1,2],[3,9],[5,6],[7,8]]
print(f"  新列表: {new_shallow}")  # 输出：[[1,2],[3,9],[5,6]]（嵌套被修改）
print(f"  嵌套元素内存地址是否相同: {id(old_list[1]) == id(new_shallow[1])}")  # 输出：True

# 重置原列表
old_list = [[1, 2], [3, 4], [5, 6]]

# 3. 深拷贝（copy.deepcopy）：全层级复制，完全独立
new_deep = copy.deepcopy(old_list)
# 修改顶层元素（新列表不受影响）
old_list.append([7, 8])
# 修改嵌套元素（新列表不受影响，嵌套列表是新对象）
old_list[1][1] = 9
print("\n3. 深拷贝:")
print(f"  原列表: {old_list}")  # 输出：[[1,2],[3,9],[5,6],[7,8]]
print(f"  新列表: {new_deep}")  # 输出：[[1,2],[3,4],[5,6]]（完全独立）
print(f"  嵌套元素内存地址是否相同: {id(old_list[1]) == id(new_deep[1])}")  # 输出：False

## 5. 列表推导式（对应课件🔶2-422至）
列表推导式是Python简化“生成新列表”的语法糖，将“for循环+条件判断”浓缩为一行，核心语法：  
`[expression for elem in iterable if test]`  
- `expression`：对元素的操作（如平方、取长度、类型转换）；
- `iterable`：待迭代的序列（如列表、range、字符串）；
- `if test`（可选）：筛选元素的条件（仅满足条件的元素才参与`expression`操作）。

课件基础示例：将“生成元素平方的列表”从普通for循环（4行）简化为列表推导式（1行），代码效率一致，但可读性更高。

### 5.1 基础用法：替代普通for循环（课件重点）
```python
# 列表推导式基础：替代普通for循环（对应课件🔶2-422至）
# 普通函数：生成元素平方的列表（课件示例）
def square_list_func(L):
    Lnew = []
    for e in L:
        Lnew.append(e ** 2)
    return Lnew

# 列表推导式：一行实现相同功能
L = [1, 2, 3, 4, 5]
L_square_func = square_list_func(L)
L_square_comp = [e ** 2 for e in L]  # 推导式：e**2（操作）+ for e in L（迭代）

print("普通函数结果:", L_square_func)  # 输出：[1, 4, 9, 16, 25]
print("列表推导式结果:", L_square_comp)  # 输出：[1, 4, 9, 16, 25]

# 带条件筛选：仅偶数平方（课件🔶2-441至）
L_even_square = [e ** 2 for e in L if e % 2 == 0]
print("带条件推导式（偶数平方）:", L_even_square)  # 输出：[4, 16]

In [ ]:
# 列表推导式进阶拓展（课件未覆盖，补充核心用法）
import sys

# 1. 嵌套列表推导式：处理二维列表（矩阵）
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
# 需求1：二维列表转一维列表（等同于 [e for row in matrix for e in row]）
flattened = [e for row in matrix for e in row]
print("\n嵌套推导式（二维转一维）:", flattened)  # 输出：[1,2,3,4,5,6,7,8,9]

# 需求2：矩阵每个元素平方
matrix_square = [[e ** 2 for e in row] for row in matrix]
print("嵌套推导式（矩阵元素平方）:", matrix_square)  # 输出：[[1,4,9],[16,25,36],[49,64,81]]

# 2. 生成器表达式：节省内存（对比列表推导式）
# 生成100万个数的平方（列表推导式会占用大量内存）
list_comp = [e ** 2 for e in range(1000000)]
gen_expr = (e ** 2 for e in range(1000000))  # 生成器表达式

# 对比内存占用（sys.getsizeof返回对象本身的内存，列表推导式存储所有元素，生成器仅存储状态）
print(f"\n列表推导式内存占用: {sys.getsizeof(list_comp)} 字节")  # 约8MB（存储100万int）
print(f"生成器表达式内存占用: {sys.getsizeof(gen_expr)} 字节")  # 约24字节（仅存储迭代状态）

# 生成器的使用：通过for循环迭代（按需计算）
print("生成器前5个元素:", [next(gen_expr) for _ in range(5)])  # 输出：[0,1,4,9,16]

In [ ]:
# 课件列表推导式练习（对应课件🔶2-462至）
# 需求：计算列表中所有字符串元素的长度，忽略非字符串元素
elements = ['xy', 'abcd', 7, '4.0']
# 推导式拆解：
# - 迭代：for x in elements
# - 条件：if type(x) == str（仅保留字符串）
# - 操作：len(x)（计算字符串长度）
str_lengths = [len(x) for x in elements if type(x) == str]

print("输入列表:", elements)
print("字符串长度列表（筛选str类型）:", str_lengths)  # 输出：[2, 4, 3]（符合课件期望）

【额外拓展】：若需同时处理“字符串子类”（如`class MyStr(str): pass`），需用`isinstance(x, str)`替代`type(x) == str`——`isinstance`会识别子类，`type`仅判断严格类型匹配。课件场景中无子类，两种写法均可。

## 6. 异常处理（对应课件🔶2-513至）
当代码遇到“意外情况”（如索引越界、除以0、用户输入非数字）时，Python会抛出“异常（Exception）”并终止程序。通过`try-except`语句可捕获异常并自定义处理逻辑（如提示用户、返回默认值），避免程序崩溃——这是保障代码健壮性的核心机制。

### 6.1 常见异常类型（课件🔶2-518至）
课件明确列举4种高频异常，需掌握其触发场景：
- `IndexError`：列表索引超出范围（如`test = [1,2], test[3]`）；
- `TypeError`：操作或函数应用于不适当类型（如`'a' / 4`、`int('abc')`）；
- `NameError`：引用未定义的变量（如`print(undefined_var)`）；
- `ZeroDivisionError`：除法或取模运算中除数为0（如`5 / 0`）。

【额外拓展】：其他常用异常类型（课件未覆盖，但开发中高频）：
- `ValueError`：值不符合预期（如`int('12a')`、`float('abc')`，区别于`TypeError`的“类型不匹配”）；
- `KeyError`：字典中查找不存在的键（如`d = {'a':1}, d['b']`）；
- `FileNotFoundError`：打开不存在的文件（如`open('nonexistent.txt')`）。

In [ ]:
# 常见异常类型示例（对应课件🔶2-518至，补充拓展类型）
def test_common_exceptions():
    # 1. IndexError：索引越界
    try:
        test_list = [1, 2]
        print(test_list[3])
    except IndexError as e:
        print(f"\n1. IndexError: {e}")  # 输出：list index out of range

    # 2. TypeError：类型不匹配
    try:
        print('a' / 4)
    except TypeError as e:
        print(f"2. TypeError: {e}")  # 输出：unsupported operand type(s) for /: 'str' and 'int'

    # 3. NameError：变量未定义
    try:
        print(undefined_var)
    except NameError as e:
        print(f"3. NameError: {e}")  # 输出：name 'undefined_var' is not defined

    # 4. ZeroDivisionError：除以0
    try:
        print(5 / 0)
    except ZeroDivisionError as e:
        print(f"4. ZeroDivisionError: {e}")  # 输出：division by zero

    # 拓展：ValueError：值不符合预期
    try:
        print(int('12a'))
    except ValueError as e:
        print(f"5. ValueError: {e}")  # 输出：invalid literal for int() with base 10: '12a'

test_common_exceptions()

In [ ]:
# 基础异常处理：try-except（对应课件🔶2-530至）
def sum_digits(s):
    """
    课件示例：计算字符串中所有数字字符的和
    用try-except捕获非数字字符的异常
    """
    total = 0
    for char in s:
        try:
            # 可能抛出ValueError的代码（非数字字符无法转int）
            val = int(char)
            total += val
        except ValueError:
            # 自定义异常处理：提示非数字字符，继续下一个字符
            print(f"警告：跳过非数字字符 '{char}'")
    return total

# 测试（含非数字字符）
test_str1 = "123a45"
print(f"\nsum_digits('{test_str1}') = {sum_digits(test_str1)}")  # 输出：1+2+3+4+5=15

# 测试（全非数字字符）
test_str2 = "abcde"
print(f"sum_digits('{test_str2}') = {sum_digits(test_str2)}")  # 输出：0

In [ ]:
# 进阶异常处理：特定异常+finally+else（对应课件🔶2-553至）
def safe_divide_with_cleanup():
    """
    处理用户输入除法，用finally确保资源清理（如关闭文件）
    用else处理正常逻辑，用多个except区分异常类型
    """
    file = None  # 初始化文件对象
    try:
        # 1. 尝试获取用户输入并计算
        a = int(input("请输入被除数: "))
        b = int(input("请输入除数: "))
        result = a / b
        
        # 2. 尝试打开文件（模拟资源操作）
        file = open("divide_result.txt", "w")
        file.write(f"{a}/{b} = {result}")
        
        # 3. try无异常时执行else
        print(f"计算成功！结果: {a}/{b} = {result}")
        return result
    
    except ValueError:
        print("错误：输入必须是整数！")
    except ZeroDivisionError:
        print("错误：除数不能为0！")
    except Exception as e:
        # 捕获其他未预料的异常（避免程序崩溃）
        print(f"未知错误: {e}")
    finally:
        # 无论是否异常，都关闭文件（资源清理）
        if file:
            file.close()
            print("文件已关闭（finally确保）")

# 测试（可手动输入不同值，如"abc"、0、5和2）
# safe_divide_with_cleanup()  # 取消注释运行

In [ ]:
# 主动抛异常：raise（对应课件🔶2-573至）
def sum_digits_strict(s):
    """
    严格版sum_digits：
    - 空字符串抛AssertionError
    - 非数字字符抛ValueError
    """
    # 用assert验证输入非空（课件🔶2-586至的断言用法）
    assert len(s) != 0, "错误：输入字符串不能为空！"
    
    total = 0
    for char in s:
        if not char.isdigit():
            # 主动抛ValueError，明确错误原因
            raise ValueError(f"错误：字符串包含非数字字符 '{char}'（仅允许0-9）")
        total += int(char)
    return total

# 测试1：正常输入
try:
    print(f"\nsum_digits_strict('1234') = {sum_digits_strict('1234')}")  # 输出：10
except (AssertionError, ValueError) as e:
    print(e)

# 测试2：空字符串（触发assert）
try:
    sum_digits_strict('')
except AssertionError as e:
    print(f"捕获assert错误: {e}")  # 输出：错误：输入字符串不能为空！

# 测试3：含非数字字符（主动抛ValueError）
try:
    sum_digits_strict('12a3')
except ValueError as e:
    print(f"捕获主动抛出的错误: {e}")  # 输出：错误：字符串包含非数字字符 'a'（仅允许0-9）

## 7. 断言（Assertion）：防御性编程工具（对应课件🔶2-586至）
断言用于“验证程序运行时的假设”（如“输入列表非空”“两个列表长度相等”），语法：  
`assert <condition>, "错误提示信息"`  
- 若`condition`为`True`，程序继续执行；
- 若`condition`为`False`，抛出`AssertionError`并终止程序，同时打印“错误提示信息”。

### 7.1 核心用途（课件重点）
- **检查输入合法性**：如函数参数是否符合约束（如`sum_digits_strict`中“输入字符串非空”）；
- **避免错误传播**：在错误早期终止程序，避免后续代码因错误输入产生更复杂的问题（如“无成绩学生的平均分计算”）；
- **文档化假设**：通过断言明确代码依赖的条件（如“此函数仅处理非空列表”），提升代码可读性。

【额外拓展】：断言的“局限性”——Python运行时可通过`-O`（优化）模式关闭断言（`assert`语句会被视为注释），因此**不能用断言处理用户输入错误或生产环境的异常场景**（如用户输入非数字），仅用于“开发阶段的假设验证”（如函数内部的参数检查）。

In [ ]:
# 断言示例与局限性说明（对应课件🔶2-586至）
def pairwise_div(Lnum, Ldenom):
    """
    课件练习：返回Lnum与Ldenom的逐元素除法列表
    用断言验证输入约束（开发阶段），用raise处理运行时错误（生产阶段）
    """
    # 1. 用断言验证输入假设（开发阶段，-O模式下会被关闭）
    assert len(Lnum) == len(Ldenom), "开发提示：Lnum与Ldenom长度必须相等！"
    assert len(Lnum) > 0 and len(Ldenom) > 0, "开发提示：Lnum与Ldenom不能为空！"
    
    result = []
    for num, denom in zip(Lnum, Ldenom):
        # 2. 用raise处理运行时错误（生产阶段，不受-O模式影响）
        if denom == 0:
            raise ValueError("运行时错误：Ldenom包含0，无法执行除法！")
        result.append(num / denom)
    return result

# 测试1：正常输入（断言通过，无异常）
L1 = [4, 5, 6]
L2 = [1, 2, 3]
print(f"\n pairwise_div({L1}, {L2}) = {pairwise_div(L1, L2)}")  # 输出：[4.0, 2.5, 2.0]

# 测试2：长度不相等（触发断言）
try:
    L3 = [4, 5]
    L4 = [1, 2, 3]
    pairwise_div(L3, L4)
except AssertionError as e:
    print(f"捕获断言错误: {e}")  # 输出：开发提示：Lnum与Ldenom长度必须相等！

# 测试3：含0（主动抛ValueError）
try:
    L5 = [4, 5, 6]
    L6 = [1, 0, 3]
    pairwise_div(L5, L6)
except ValueError as e:
    print(f"捕获运行时错误: {e}")  # 输出：运行时错误：Ldenom包含0，无法执行除法！

## 8. 综合示例：学生成绩统计（异常与断言的实战应用）（对应课件🔶2-603至）
课件用“学生成绩统计”展示异常与断言的综合应用，需求：“给每个学生的成绩列表添加平均分，处理‘无成绩’的情况”。以下基于课件逻辑，补充“成绩范围检查”和“批量班级处理”的拓展，展示代码的可扩展性。
```python
# 综合示例：学生成绩统计（含异常、断言、批量处理）（对应课件🔶2-603至）
def validate_grade(grade):
    """拓展：验证成绩是否在0-100之间，超出则抛ValueError"""
    if not (0 <= grade <= 100):
        raise ValueError(f"成绩 {grade} 无效（必须在0-100之间）")
    return grade

def calculate_avg(grades, policy="halt"):
    """
    计算平均分，支持三种策略（课件基础上补充成绩验证）
    policy: "halt"（断言终止）、"zero"（返回0）、"warn"（警告+返回None）
    """
    # 1. 验证成绩合法性（拓展：检查每个成绩是否在0-100）
    for grade in grades:
        validate_grade(grade)
    
    # 2. 处理无成绩情况（课件核心逻辑）
    if policy == "halt":
        assert len(grades) != 0, "无成绩数据，无法计算平均分！"
        return sum(grades) / len(grades)
    elif policy == "zero":
        try:
            return sum(grades) / len(grades)
        except ZeroDivisionError:
            print("警告：无成绩，平均分按0计算")
            return 0.0
    elif policy == "warn":
        try:
            return sum(grades) / len(grades)
        except ZeroDivisionError:
            print("警告：无成绩，无法计算平均分")
            return None
    else:
        raise ValueError("policy仅支持halt/zero/warn")

def process_class_grades(class_list, policy="halt"):
    """处理单个班级的成绩，添加平均分（课件逻辑）"""
    processed = []
    for student in class_list:
        # 假设学生数据格式：[姓名列表, 成绩列表]（如[['peter', 'parker'], [80.0, 70.0, 85.0]]）
        assert len(student) == 2, "学生数据格式错误：必须包含姓名和成绩！"
        name, grades = student
        try:
            avg_grade = calculate_avg(grades, policy)
            processed.append([name, grades, round(avg_grade, 2) if avg_grade is not None else None])
        except (AssertionError, ValueError) as e:
            print(f"处理学生 {name} 时出错: {e}")
            processed.append([name, grades, "计算失败"])
    return processed

def process_multiple_classes(classes, policy="halt"):
    """拓展：批量处理多个班级的成绩"""
    all_processed = []
    for class_name, class_list in classes.items():
        print(f"\n=== 处理班级：{class_name} ===")
        processed_class = process_class_grades(class_list, policy)
        all_processed.append({class_name: processed_class})
    return all_processed

# 测试数据：两个班级，含无成绩、无效成绩的学生（课件数据基础上拓展）
test_classes = {
    "Class A": [
        [['peter', 'parker'], [80.0, 70.0, 85.0]],    # 正常成绩
        [['bruce', 'wayne'], [100.0, 80.0, 74.0]],   # 正常成绩
        [['deadpool'], []],                           # 无成绩
        [['tony', 'stark'], [95.0, 105.0, 88.0]]     # 无效成绩（105超出范围）
    ],
    "Class B": [
        [['steve', 'rogers'], [85.0, 90.0, 82.0]],    # 正常成绩
        [['natasha', 'romanoff'], []]                 # 无成绩
    ]
}

# 批量处理所有班级（策略：zero，无成绩按0计算）
processed_classes = process_multiple_classes(test_classes, policy="zero")

# 打印结果
print("\n=== 批量处理结果 ===")
for cls in processed_classes:
    for class_name, students in cls.items():
        print(f"\n{class_name}:")
        for stu in students:
            name, grades, avg = stu
            print(f"  {name}: 成绩={grades}, 平均分={avg}")

## 9. 核心总结（对应课件🔶2-400至）
1. **迭代修改列表**：
   - 难点：for循环迭代器的计数器不随列表修改更新，导致元素跳过；
   - 解决方案：迭代克隆列表（`L[:]`/`copy.copy`），修改原列表；
   - 拓展：用`enumerate`跟踪迭代索引，辅助调试。

2. **深浅拷贝**：
   - 浅拷贝（`L[:]/copy.copy`）：仅复制顶层，适合不可变元素列表；
   - 深拷贝（`copy.deepcopy`）：复制所有层级，适合可变嵌套元素；
   - 拓展：用`id()`验证内存地址，区分别名与拷贝。

3. **列表推导式**：
   - 基础：`[expr for elem in iterable if test]`，简化新列表生成；
   - 拓展：嵌套推导式处理二维数据，生成器表达式节省内存。

4. **异常处理**：
   - 核心：`try-except`捕获异常，`raise`主动抛异常；
   - 拓展：`finally`清理资源，`else`处理正常逻辑，生产环境不用断言处理用户输入。

5. **断言**：
   - 用途：开发阶段验证假设（如输入非空），文档化代码约束；
   - 局限：`-O`模式下关闭，不能替代异常处理生产环境错误。

6. **列表vs元组**：
   - 列表：可变，适合频繁修改（如成绩更新），效率高；
   - 元组：不可变，适合保护核心数据（如学生ID），避免意外修改。